
## Imports

In [1]:
%reload_ext autoreload
%autoreload 2
from merkle_tree import MerkleTree
from channel import Channel
from hash import my_hash
from pprint import pprint

In [2]:
# Test

from pprint import pprint

m = MerkleTree([1, 2, 3, 4, 5, 6, 7, 8])
pprint([[b[:4] for b in a] for a in m.tree])
print(m.root)
pprint([a[:4] for a in m.get_path(0)])

[['6b86', 'd473', '4e07', '4b22', 'ef2d', 'e7f6', '7902', '2c62'],
 ['33b6', '1365', '4358', 'ada1'],
 ['85df', 'e0e2'],
 ['c274']]
c27450cd3fd4df029145f3437ae9c381e0ae55e8400de06cb973005b36d7b222
['6b86',
 ('d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35', 'right'),
 ('13656c83d841ea7de6ebf3a89e0038fea9526bd7f686f06f7a692343a8a32dca', 'right'),
 ('e0e2d0cec0ef7e8fc458e516dfde82890c183431a3f9efae9e4693fc23dfa36a', 'right')]


### Fibonacci function

In [3]:
def fibonacci(a=1, size=1001) -> list[int]:
    fib_list = [1, a]
    for _ in range(2, size):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list

In [4]:
print(fibonacci())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777836

### Polynomial f(x)

In [5]:
P: int =3221225473
F = GF(P)
N = 8192

g = F(1734477367)
gamma = g ** 8
gamma_group = [gamma ** i for i in range(1001)]

fibonacci_seq = fibonacci()

R.<x> = PolynomialRing(F)

f = R.lagrange_polynomial(zip(gamma_group, fibonacci_seq))

### Bit Reverse Order

In [23]:
def bit_reverse(n, width):
    rev = 0  # This will store the reversed bits
    for i in range(width):
        rev = (rev << 1) | (n & 1)  # Shift left and take LSB of n
        n >>= 1  # Shift n to the right
    return rev


def bit_reverse_permutation(N):
    """Generate bit-reversed order for N elements."""
    width = int(math.log(N, 2))
    return [bit_reverse(i, width) for i in range(N)]


bit_reverse_order = bit_reverse_permutation(N)

### w < g >

In [7]:
w = F.multiplicative_generator()

g_group = [g ** i for i in range(N)]

wg = [w * gi for gi in g_group]
wg_merkle_order = [wg[i] for i in bit_reverse_order]

### Commit f(x) on LDE

In [8]:
channel = Channel(F)

f_wg = [f(x) for x in wg_merkle_order]

merkle_f_wg = MerkleTree(f_wg)

channel.send({'title': 'f(x) on LDE', 'data': merkle_f_wg.root})

### Polynomial h(x)

In [9]:
d_x = 1
for gamma_i in gamma_group[:-2]:
    d_x = d_x * (x - gamma_i)

F_x = f(x) + f(gamma * x) - f((gamma**2) * x)

h_x = F_x // d_x

In [10]:
h_wg = [F_x(x) / d_x(x) for x in wg_merkle_order]

# Sanity check
h_wg_by_poly = [h_x(x) for x in wg_merkle_order]
print(h_wg_by_poly[:10])
print(h_wg[:10])
assert(h_wg == h_wg_by_poly)

[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]
[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]


### Boundary Constraints

In [11]:
Y = fibonacci_seq[-1]

t0_x = (f(x) - 1) / (x - gamma ** 0)
t1_x = (f(x) - Y) / (x - gamma ** 1000)

In [12]:
t0_wg = [(f(i) - 1) / (i - gamma ** 0) for i in wg_merkle_order]
t1_wg = [(f(i) - Y) / (i - gamma ** 1000) for i in wg_merkle_order]

# Sanity check
t0_x_by_poly = [t0_x(x) for x in wg_merkle_order]
t1_x_by_poly = [t1_x(x) for x in wg_merkle_order]
print(t0_x_by_poly[:10])
print(t0_wg[:10])
print("")
print(t1_x_by_poly[:10])
print(t1_wg[:10])
assert(t0_wg == t0_x_by_poly)
assert(t1_wg == t1_x_by_poly)

[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]
[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]

[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]
[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]


### Composition Polynomial

In [13]:
beta_0 = channel.receive_random_field_element()
beta_1 = channel.receive_random_field_element()
beta_2 = channel.receive_random_field_element()

cp0_x = beta_0 * h_x + beta_1 * t0_x + beta_2 * t1_x

In [14]:
cp0_wg = [beta_0 * h_wg[i] + beta_1 * t0_wg[i] + beta_2 * t1_wg[i]
          for i in range(len(wg))]

# Sanity check
cp0_x_by_poly = [cp0_x(x) for x in wg_merkle_order]
print(cp0_x_by_poly[:10])
print(cp0_wg[:10])
assert(cp0_wg == cp0_x_by_poly)

[889138553, 3220141988, 2677341730, 1561002692, 987928056, 1023021936, 2026596108, 479199681, 2909342300, 288377119]
[889138553, 3220141988, 2677341730, 1561002692, 987928056, 1023021936, 2026596108, 479199681, 2909342300, 288377119]


### FRI

In [15]:
fri_layers = []
degree = R(cp0_x).degree()
# if degree % 2 != 0:
#     degree += 1

curr_cp = [cp0_x(x) for x in wg_merkle_order]
while degree > 0:
    # Commit curr layer
    merkle_curr_cp = MerkleTree(curr_cp)
    fri_layers.append((curr_cp, merkle_curr_cp))
    channel.send({
        "title": f"commit for layer {len(fri_layers) - 1} of FRI",
        "data": merkle_curr_cp.root,
    })
    random_x = channel.receive_random_field_element()
    curr_cp = [((curr_cp[i] + curr_cp[i + 1]) / 2) +
               random_x * (curr_cp[i] - curr_cp[i + 1]) / (2 * wg_merkle_order[i]) for i in range(len(curr_cp))[::2]]
    degree = degree // 2

constant = curr_cp[0]
channel.send({"title:": "constant of last FRI layer",
              "data": str(constant),
              })

assert(all([constant == x for x in curr_cp]))

In [16]:
print(fri_layers)
print("Merkle Root")
print(curr_cp[0])

[([889138553, 3220141988, 2677341730, 1561002692, 987928056, 1023021936, 2026596108, 479199681, 2909342300, 288377119, 1849257519, 848818080, 479524676, 489539242, 2051835259, 53040003, 2535585294, 898462144, 2321870485, 1355650170, 2547297337, 1943581321, 1694803032, 2608521411, 2097165828, 2564419897, 2655045814, 2317115005, 1377495246, 1853213449, 1086161249, 2418374382, 54124744, 2562120488, 1241785964, 1109978641, 982726136, 2456692, 625549458, 37265863, 2945393824, 2264013016, 2614975549, 402474916, 1128987817, 1704671230, 54795704, 632202085, 1550739130, 3106851322, 1505189074, 2356599163, 2738229460, 2647346594, 2460026065, 2528029858, 1425850109, 2168484011, 2744048771, 2159860731, 1396374378, 899931650, 467882401, 2092287857, 1999268405, 2326699804, 2822983317, 669254928, 1406278078, 3109638431, 1359806360, 2053134989, 49205464, 1726598182, 3094942630, 304168601, 1328232562, 2163133396, 1467734349, 2360600004, 2999550560, 216737703, 3132243467, 1367424703, 569131423, 31173240

In [17]:
# Sanity check
print(curr_cp)

[2347139236, 2347139236, 2347139236, 2347139236, 2347139236, 2347139236, 2347139236, 2347139236]


In [18]:
pprint(channel.get_all_messages())

[{'data': 'd524fba68e1263f084afe501c715dca773c616d5a8c52303322985d32c94ea56',
  'title': 'f(x) on LDE'},
 {'data': '2415762890', 'title': 'Random Field Element'},
 {'data': '2203155640', 'title': 'Random Field Element'},
 {'data': '2854904680', 'title': 'Random Field Element'},
 {'data': 'edbd0af38477fe79ef91e8f837346ece4032f8efecaf6be8317635d062eeacd3',
  'title': 'commit for layer 0 of FRI'},
 {'data': '2547207347', 'title': 'Random Field Element'},
 {'data': 'c71e67cff71e25df41a9a8a9b144cdee9a85707a4011cc68c9b5816f4fdc4586',
  'title': 'commit for layer 1 of FRI'},
 {'data': '676878935', 'title': 'Random Field Element'},
 {'data': '614a8be63d00236efe2661c852c7edec98fc8ae8d88842d4b3451a5068de128b',
  'title': 'commit for layer 2 of FRI'},
 {'data': '2252690546', 'title': 'Random Field Element'},
 {'data': 'b46eb83a04af524cac1dcd79fab88b93e9e654f030194bdb148bfcd0ddc3fef1',
  'title': 'commit for layer 3 of FRI'},
 {'data': '2775482430', 'title': 'Random Field Element'},
 {'data': '257

### Decommit Phase

# TIOTA


## Decommit Phase

In [19]:
def send_f_of_x_and_gammas(idx_of_x, idx_of_gamma_x, idx_of_gamma_2_x):
    channel.send({
        'title': 'f of x',
        'data': f_wg[idx_of_x],
    })
    channel.send({
        'title': 'path of f of x',
        'data': merkle_f_wg.get_path(idx_of_x),
    })
    channel.send({
        'title': 'f of gamma x',
        'data': f_wg[idx_of_gamma_x],
    })
    channel.send({
        'title': 'path of f of gamma x',
        'data': merkle_f_wg.get_path(idx_of_gamma_x),
    })
    channel.send({
        'title': 'f of gamma squared x',
        'data': f_wg[idx_of_gamma_2_x],
    })
    channel.send({
        'title': 'path of f of gamma squared x',
        'data': merkle_f_wg.get_path(idx_of_gamma_2_x),
    })

def send_cp_layer(idx_of_x, idx_of_minus_x, cp, cp_merkle, i):
    assert(cp_merkle.get_path(idx_of_x)[2:] == cp_merkle.get_path(idx_of_minus_x)[2:])
    channel.send({ # DEBUG
        'title': f'cp of x - layer {i}',
        'data': cp[idx_of_x],
    })
    channel.send({
        'title': f'path of cp of x - layer {i}',
        'data': cp_merkle.get_path(idx_of_x),
    })
    channel.send({
        'title': f'cp of -x - layer {i}',
        'data': cp[idx_of_minus_x],
    })
    channel.send({
        'title': f'path of cp of -x - layer {i}',
        'data': cp_merkle.get_path(idx_of_minus_x),
    })

def query(idx):
    idx_of_x = bit_reverse_order.index(idx)
    idx_of_gamma_x = bit_reverse_order.index(idx + 8)
    idx_of_gamma_2_x = bit_reverse_order.index(idx + 16)
    send_f_of_x_and_gammas(idx_of_x, idx_of_gamma_x, idx_of_gamma_2_x)
    for i, (cp, cp_merkle) in enumerate(fri_layers):
        idx_of_minus_x = idx_of_x ^^ 1
        send_cp_layer(idx_of_x, idx_of_minus_x, cp, cp_merkle, i)
        idx_of_x //= 2

In [20]:
for i in range(8):
    random_idx = int(channel.receive_random_field_element()) % len(wg)
    query(random_idx)

In [21]:
def check_merkle_path(data, path, root):
    curr = my_hash(data)
    assert(curr == path[0])
    for sibling, direction in path[1:]:
        if direction == 'left':
            curr = my_hash(sibling + curr)
        else:
            curr = my_hash(curr + sibling)
    assert curr == root

pprint(merkle_f_wg.get_path(4000))
check_merkle_path(f_wg[4000], merkle_f_wg.get_path(4000), merkle_f_wg.root)

['4b6943e02db40109d4ed016a4b9701c82d0078563b763cf71e70ef07d8e1ae8b',
 ('4fb51ad90b1d1328a75a1d7d199d8442d456674287d9275e2eb80da14e5bc838', 'right'),
 ('429afab6d0d4897b99e0a7357df624e02d4d86582d528cf66461e9dd16ef34ff', 'right'),
 ('3bc1d36482529647a772156b995e7d4b45d1650f498edb4cfe3bd748518fe472', 'right'),
 ('7b9a037629519e3790aa8de19f0da68df3bd24732aec1f10c1214b6eb97a5137', 'right'),
 ('37842539e2d1555d69b021802fdd59e2e680c2a70fbe04792f64429038480a24', 'right'),
 ('fbdecc40f5213c06186be7d3803d634bed31e694f52f66f3a28e5c3285fd3a1c', 'left'),
 ('f316f1a93f7246fec41d8bdbe90b6932bfe23eb9b5382a3d7a63e810d79e0d93', 'right'),
 ('ed00fd01cedf98a0a6612003f2f4f8d92aa65150e959697e6629bd16bb245aa0', 'left'),
 ('af8f6f5805c190a9f6876e5ed431f1fe89a6f389dc81c189dd4ab5d63bb33ab0', 'left'),
 ('a49f0ede6a8fa1715a1c005795fa36fbaf564ecd0b55c7267a2461232c10e78e', 'left'),
 ('56724ef5607984e5ff656ea0f36354a8c70e3640382aa831ae22b7a15b4789f0', 'left'),
 ('21d3fdd36a75a8e6cd5693c24f60c8682fc2c4deaddf8d303ba1c

In [22]:
from random import randint, seed
seed(int(42069))

rand_idx = randint(0, len(wg))
rand_x = wg[rand_idx]
f_x_at_idx = f(rand_x)
f_x_at_g_idx = f(gamma * rand_x)
f_x_at_g2_idx = f(gamma**2 * rand_x)

channel.send({'title': 'decommit phase 0 ', 'data': {
             'result': f_x_at_idx, 'path': merkle_f_wg.get_path(rand_idx)}})

for i in range(len(cps)):
    cp = cps[i]
    cp_at_idx = cp(rand_x)
    cp_at_neg_idx = cp(-rand_idx)

NameError: name 'cps' is not defined